In [182]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.python.client import device_lib
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow
import numpy as np

In [183]:
def readDatasets(path):
    f = open(path, 'r', encoding='utf-8')
    texts = f.readlines()
    f.close()
    return texts

text_train = readDatasets('HerMajestySpeechesDataset/train.txt')
text_test = readDatasets('HerMajestySpeechesDataset/test.txt')
text_val = readDatasets('HerMajestySpeechesDataset/dev.txt')



In [376]:
tokenizer_train = Tokenizer(oov_token='<unk>')
tokenizer_train.fit_on_texts(text_train) 
texts2ids_train = tokenizer_train.texts_to_sequences(text_train)

texts2ids_test = tokenizer_train.texts_to_sequences(text_test)

texts2ids_val = tokenizer_train.texts_to_sequences(text_val)

# print ("Texts as IDs:", texts2ids[:10])
# ids2texts = tokenizer_train.sequences_to_texts(texts2ids)
# print ("IDs back to texts:", ids2texts[:10])

In [365]:
tokenizer_train.sequences_to_texts([[0]])

['<unk>']

In [238]:
np.max(np.concatenate(texts2ids_train))

5614.0

In [239]:
print(len(texts2ids_train[0]), len(texts2ids_test[0]), len(texts2ids_val[0]))

12 8 13


In [240]:
for i in texts2ids_train:
    i.append(5615)

for i in texts2ids_test:
    i.append(5615)

for i in texts2ids_val:
    i.append(5615)

In [241]:
print(len(texts2ids_train[0]), len(texts2ids_test[0]), len(texts2ids_val[0]))


13 9 14


In [242]:
def get_ngrams(frase, size=2):
    for i in range(size):
        frase = [0] + frase
    ngrams_list = []
    for i in range(len(frase)-size):
        ngrams_list.append((tuple(frase[i:i+size]),frase[i+size]))
    return ngrams_list

all_ngrams = []
for i in texts2ids_train:
    all_ngrams += get_ngrams(i)

In [254]:
def co_table(lista_ocurrencias):
    table = {}
    for i in lista_ocurrencias:
        if i[0] in table:
            if i[1] in table[i[0]]:
                table[i[0]][i[1]] += 1
            else:
                table[i[0]][i[1]] = 1
        else:
            table[i[0]] = {}
            table[i[0]][i[1]] = 1
    return table

table = co_table(all_ngrams)

{(0, 0): {2: 237, 21: 17, 68: 17, 8: 256, 58: 9, 17: 148, 15: 58, 874: 2, 4: 69, 881: 2, 80: 6, 11: 42, 7: 34, 6: 73, 132: 26, 233: 9, 73: 24, 22: 32, 9: 83, 291: 3, 14: 116, 2126: 1, 100: 4, 1657: 2, 67: 32, 94: 1, 121: 11, 20: 13, 1180: 1, 2135: 1, 382: 3, 601: 5, 26: 21, 18: 32, 10: 26, 40: 68, 3030: 1, 25: 36, 743: 1, 157: 1, 240: 8, 16: 28, 31: 10, 1687: 1, 207: 6, 5615: 5, 406: 2, 338: 9, 83: 13, 27: 11, 62: 23, 48: 28, 613: 1, 51: 22, 150: 6, 140: 16, 174: 13, 46: 8, 98: 1, 99: 3, 47: 7, 60: 23, 190: 14, 79: 2, 159: 1, 752: 1, 3110: 1, 913: 3, 2220: 1, 54: 4, 113: 7, 93: 8, 1232: 1, 92: 1, 555: 1, 43: 5, 1012: 1, 758: 1, 1427: 1, 215: 1, 1430: 4, 3185: 1, 50: 3, 39: 5, 28: 5, 71: 3, 2255: 2, 144: 6, 42: 1, 1078: 3, 1054: 3, 1414: 2, 41: 4, 3239: 1, 674: 1, 1455: 1, 160: 1, 142: 2, 23: 9, 344: 2, 296: 2, 91: 7, 813: 1, 106: 5, 2310: 1, 842: 1, 5: 21, 1099: 1, 37: 1, 1483: 1, 203: 4, 103: 5, 1196: 1, 466: 1, 82: 3, 2353: 1, 848: 5, 449: 5, 196: 3, 399: 11, 288: 2, 3433: 1, 453: 4,

In [245]:
def generate_toyLM_ngram_a(table, context='aleatorio', n=15):
    if context == 'aleatorio':
        tmp = list(table.keys())
        context = tmp[np.random.randint(len(tmp))]

    cadena = list(context)
    for _ in range(n-len(context)):
        context = tuple(cadena[-len(context):])
        if context not in table.keys():
            break
        else:
            new = max(table[context], key=table[context].get)
            if new == 5615:
                break
            cadena.append(new)
    return tokenizer_train.sequences_to_texts([cadena])

generate_toyLM_ngram_a(table)


5615


['or terror for the future']

In [253]:
def generate_toyLM_ngram_b(table, context='aleatorio', n=15):
    if context == 'aleatorio':
        tmp = list(table.keys())
        context = tmp[np.random.randint(len(tmp))]

    cadena = list(context)
    for i in range(n-len(context)):
        context = tuple(cadena[-len(context):])
        if context not in table.keys():
            break
        else:
            lista_tmp = list()
            for i in table[context]:
                for j in range(table[context][i]):
                    lista_tmp.append(i)
            new = np.random.randint(len(lista_tmp))
            if new == 5615:
                break
            cadena.append(lista_tmp[new])
    return tokenizer_train.sequences_to_texts([cadena])

generate_toyLM_ngram_b(table)


4


['process that starts within the twenty first century <unk>']

# toyLM_LSTM

In [273]:
def train_generate(text, max_seq_length=10):
    train_set = dict()
    for frase in text:
        for word_index in range(len(frase)):
            if word_index < max_seq_length:
                train_set[tuple(pad_sequences([frase[:word_index]], maxlen=max_seq_length)[0])] = frase[word_index]
    return train_set
    
train_set = train_generate(texts2ids_train)
test_set = train_generate(texts2ids_test)
val_set = train_generate(texts2ids_val)

In [339]:
def train_generate(text, size=10):
    x = []
    y = []
    for phrase in text:
        for context, following in get_ngrams(phrase,size):
            x.append(list(context))
            y.append(following)
    return np.array(x), to_categorical(np.array(y))

In [346]:
x_train, y_train = train_generate(texts2ids_train, 2)
x_test, y_test = train_generate(texts2ids_test, 2)
x_val, y_val = train_generate(texts2ids_val, 2)

print(x_val.shape, y_val.shape)

(12405, 2) (12405, 5613)


In [315]:
x_train = np.array(list(train_set.keys()))
x_train
# y_train = to_categorical(list(train_set.values()), num_classes=len(tokenizer_train.word_index))
#
# x_test = np.array(list(test_set.keys()))
# y_test = to_categorical(list(test_set.values()), num_classes=len(tokenizer_test.word_index))
#
# x_val = np.array(list(val_set.keys()))
# y_val = to_categorical(list(val_set.values()), num_classes=len(tokenizer_val.word_index))

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   2],
       [  0,   0,   0, ...,   0,   2, 181],
       ...,
       [  0,   0,   0, ...,   5,   2, 123],
       [  0,   0, 132, ...,   2, 123, 168],
       [  0, 132, 131, ..., 123, 168,   4]])

In [347]:
model = Sequential([
    Embedding(5615, 20),
    LSTM(64),
    Dense(5615, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(x_train, y_train, batch_size=64, epochs=10)

Epoch 1/10
824/824 [==============================] - 8s 8ms/step - loss: 6.7846 - accuracy: 0.0621   

In [380]:
def generate_toyLM_lstm_a(model, context='aleatorio', n=15):
    if context=='aleatorio':
        context = list(np.random.randint(0, 5616,2))

    cadena = context
    for _ in range(n-len(context)):
        context = cadena[-len(context):]
        new = np.argmax(model.predict(np.array([context])))
        if new == 5615:
            break
        cadena.append(new)
    return tokenizer_train.sequences_to_texts([cadena])

generate_toyLM_lstm_a(model)

['privileged carol of the commonwealth of the commonwealth of the commonwealth of the commonwealth of']

In [386]:
def generate_toyLM_lstm_b(model, context='aleatorio', n=15):
    if context=='aleatorio':
        context = list(np.random.randint(0, 5616,2))

    cadena = context
    for _ in range(n-len(context)):
        context = cadena[-len(context):]
        probs = model.predict(np.array([context]))
        new = np.random.choice(range(5615), p=probs[0])
        if new == 5615:
            break
        cadena.append(new)
    return tokenizer_train.sequences_to_texts([cadena])

generate_toyLM_lstm_b(model)

['perfection acts of provincial concerns in our kind of life who have creed of democratic']

In [ ]:
import keras.backend as K

def perplexity(y_true, y_pred):
    cross_entropy = K.categorical_crossentropy(y_true, y_pred)
    perplexity = K.pow(2.0, cross_entropy)
    return perplexity

In [ ]:
y_pred = model.predict(x_test)
np.mean(perplexity(y_test, y_pred))